# _SQL_ y _people analytics_
Como dijimos en el cuaderno anterior, veremos más cláusulas de _SQL_ para comprenderlo mejor. El aprendizaje de hoy será a través de una base datos con información sobre los empleados de una empresa. Al análisis de la información de los empleados de una compañía con el propósito de identificar tendencias se le llama _people analytics_. Me gusta verlo como recursos humanos en esteroides, pues el uso del análisis de datos permite tomar decisiones cruciales respecto a la conformidad de los empleados y su permanencia, entre otras cosas. Se trata de un area que cruza los recursos humanos con la tecnología. (Por cierto, no consuman esteroides, es pésimo para la salud.)

## Introducción y preparación del entorno
Para abordar preguntas de _people analytics_ usaremos algunas de las cláusulas del cuaderno anterior y otras nuevas como `GROUP BY`, `HAVING`, y `ORDER BY`.

Antes de empezar, vamos a preparar nuestro entorno. Usaremos de nuevo `sqlalchemy` y crearemos un _engine_, nuestro puente.

In [1]:
# Instalación de sqlalchemy y conexión.
!pip install sqlalchemy pandas

import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine('sqlite:///:memory:')

Ahora crearemos las tablas.

In [2]:
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS Proyectos"))
    conn.execute(text("DROP TABLE IF EXISTS Empleados"))
    conn.execute(text("DROP TABLE IF EXISTS Departamentos"))

    conn.execute(text("""
        CREATE TABLE Departamentos (
            id INTEGER PRIMARY KEY,
            nombre TEXT
        );
    """))
    conn.execute(text("""
        CREATE TABLE Empleados (
            id INTEGER PRIMARY KEY,
            nombre TEXT,
            edad INTEGER,
            salario REAL,
            departamento_id INTEGER,
            FOREIGN KEY(departamento_id) REFERENCES Departamentos(id)
        );
    """))
    conn.execute(text("""
        CREATE TABLE Proyectos (
            id INTEGER PRIMARY KEY,
            nombre TEXT
        );
    """))


Tenemos las tablas. Son tres. Se llaman `Departamentos`, `Empleados` y `Proyectos`. Ahora le añadiremos información adicional a las tablas y haremos una impresión para comprobar que todo está en orden.

In [9]:
with engine.begin() as conn:
    conn.execute(text("INSERT INTO Departamentos (nombre) VALUES ('TI'), ('Finanzas'), ('Marketing');"))
    conn.execute(text("""
    INSERT INTO Empleados (nombre, edad, salario, departamento_id) VALUES
    ('Ana', 28, 23000, 1),
    ('Luis', 32, 27000, 2),
    ('Marta', 29, 22000, 1),
    ('Pedro', 41, 32000, 2),
    ('Raúl', 24, 18000, 3),
    ('Julia', 35, 26000, 3),
    ('Carlos', 23, 19000, NULL);
    """))
    conn.execute(text("INSERT INTO Proyectos (nombre) VALUES ('Migración de datos'), ('Reestructuración'), ('Campaña de marketing');"))

# Mostrar Departamentos.
dept = pd.read_sql_query("SELECT * FROM Departamentos", engine)
print("Tabla Departamentos:")
print(dept)
print()

# Mostrar Empleados.
empl = pd.read_sql_query("SELECT * FROM Empleados", engine)
print("Tabla Empleados:")
print(empl)
print()

# Mostrar Proyectos.
proy = pd.read_sql_query("SELECT * FROM Proyectos", engine)
print("Tabla Proyectos:")
print(proy)
print()


Tabla Departamentos:
   id     nombre
0   1         TI
1   2   Finanzas
2   3  Marketing
3   4         TI
4   5   Finanzas
5   6  Marketing

Tabla Empleados:
    id  nombre  edad  salario  departamento_id
0    1     Ana    28  23000.0              1.0
1    2    Luis    32  27000.0              2.0
2    3   Marta    29  22000.0              1.0
3    4   Pedro    41  32000.0              2.0
4    5    Raúl    24  18000.0              3.0
5    6   Julia    35  26000.0              3.0
6    7  Carlos    23  19000.0              NaN
7    8     Ana    28  23000.0              1.0
8    9    Luis    32  27000.0              2.0
9   10   Marta    29  22000.0              1.0
10  11   Pedro    41  32000.0              2.0
11  12    Raúl    24  18000.0              3.0
12  13   Julia    35  26000.0              3.0
13  14  Carlos    23  19000.0              NaN

Tabla Proyectos:
   id                nombre
0   1    Migración de datos
1   2      Reestructuración
2   3  Campaña de marketing
3   4  

Ya estamos listos para comenzar con las consultas.
## Consultas
Con esta consulta agrupamos los empleados por departamento, calculamos el salario promedio y contamos cuántos empleados hay en cada área.

Con la cláusula `GROUP BY` agrupamos por departamento y con las funciones de agregación `COUNT` y `AVG` sacamos el número de empleados y su salario promedio.

In [4]:
df = pd.read_sql_query("""
SELECT d.nombre as departamento, AVG(e.salario) as salario_promedio, COUNT(e.id) as empleados
FROM Empleados e
JOIN Departamentos d ON e.departamento_id = d.id
GROUP BY d.nombre
""", engine)
print("Promedio de salario y empleados por departamento:")
print(df)
print()


Promedio de salario y empleados por departamento:
  departamento  salario_promedio  empleados
0     Finanzas           29500.0          2
1    Marketing           22000.0          2
2           TI           22500.0          2



Si quisiéramos descubrir cuáles son los departamentos con salarios promedio superiores a 25000, debemos usar `HAVING`. Con esta cláusula  filtramos resultados de funciones de agregación.

In [5]:
df = pd.read_sql_query("""
SELECT d.nombre as departamento, AVG(e.salario) as salario_promedio
FROM Empleados e
JOIN Departamentos d ON e.departamento_id = d.id
GROUP BY d.nombre
HAVING AVG(e.salario) > 25000
""", engine)
print("Departamentos con salario promedio superior a 25000:")
print(df)
print()

Departamentos con salario promedio superior a 25000:
  departamento  salario_promedio
0     Finanzas           29500.0



`ORDER BY` simplemente organiza la información de manera ascendente o descendente. En el siguiente ejemplo vamos a ordenar con base en `salario_promedio` de forma descendente.

In [6]:
df = pd.read_sql_query("""
SELECT d.nombre, AVG(e.salario) as salario_promedio
FROM Empleados e
JOIN Departamentos d ON e.departamento_id = d.id
GROUP BY d.nombre
ORDER BY salario_promedio DESC
""", engine)
print("Departamentos ordenados por salario promedio descendente:")
print(df)
print()


Departamentos ordenados por salario promedio descendente:
      nombre  salario_promedio
0   Finanzas           29500.0
1         TI           22500.0
2  Marketing           22000.0



Finalizaremos el proyecto del día de hoy con la cláusula `CASE`, la cual crea segmento en función de una condición. Funciona de manera similar a una estructura `if`, `elif` y `else` en _Python_.

In [11]:
df = pd.read_sql_query("""
SELECT nombre,
       salario,
       CASE
           WHEN salario > 25000 THEN 'Alto'
           WHEN salario > 20000 THEN 'Medio'
           ELSE 'Bajo'
       END AS categoria_salarial
FROM Empleados
""", engine)
print("Clasificación salarial por empleado:")
print(df)
print()


Clasificación salarial por empleado:
    nombre  salario categoria_salarial
0      Ana  23000.0              Medio
1     Luis  27000.0               Alto
2    Marta  22000.0              Medio
3    Pedro  32000.0               Alto
4     Raúl  18000.0               Bajo
5    Julia  26000.0               Alto
6   Carlos  19000.0               Bajo
7      Ana  23000.0              Medio
8     Luis  27000.0               Alto
9    Marta  22000.0              Medio
10   Pedro  32000.0               Alto
11    Raúl  18000.0               Bajo
12   Julia  26000.0               Alto
13  Carlos  19000.0               Bajo

